# Spotify Suggestion Model V1

## Infrastructure

In [ ]:
import sys
# !{sys.executable} -m pip install spotipy

In [ ]:
username = "agwx2"
cid = "17c11cf9271f4980a85e6aab83cba6bb" 
secret = "015c9abf126340a7981e7ba64dbe7828"
good_playlist_id = "61BmnG6d7ifmj8Dr5tuCBK"
bad_playlist_id = "53FPslFSIAGvVyERcNG4J3"
source_playlist_id = "5ghFHvZmV4FDK5YLeZQRu9"

In [ ]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
from statistics import mean, stdev
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LinearRegression

## Data Collection

In [ ]:
sp_cred = SpotifyClientCredentials(
    client_id=cid,
    client_secret=secret
)
sp = spotipy.Spotify(client_credentials_manager=sp_cred)

In [ ]:
good_playlist = sp.user_playlist(username, good_playlist_id)
good_tracks = good_playlist["tracks"]
good_songs = good_tracks["items"]

while good_tracks['next']:
    good_tracks = sp.next(good_tracks)
    for item in good_tracks["items"]:
        good_songs.append(item)

good_ids = []
for i in range(len(good_songs)):
    good_ids.append(good_songs[i]['track']['id'])
# print(len(good_ids))
# print(good_ids)
print(good_songs)

[{'added_at': '2019-03-12T06:36:54Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/agwx2'}, 'href': 'https://api.spotify.com/v1/users/agwx2', 'id': 'agwx2', 'type': 'user', 'uri': 'spotify:user:agwx2'}, 'is_local': False, 'primary_color': None, 'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2hl0xAkS2AIRAu23TVMBG1'}, 'href': 'https://api.spotify.com/v1/artists/2hl0xAkS2AIRAu23TVMBG1', 'id': '2hl0xAkS2AIRAu23TVMBG1', 'name': 'Kansas', 'type': 'artist', 'uri': 'spotify:artist:2hl0xAkS2AIRAu23TVMBG1'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'G

In [ ]:
bad_playlist = sp.user_playlist(username, bad_playlist_id)
bad_tracks = bad_playlist["tracks"]
bad_songs = bad_tracks["items"]

while bad_tracks['next']:
    bad_tracks = sp.next(bad_tracks)
    for item in bad_tracks["items"]:
        bad_songs.append(item)

bad_ids = []
for i in range(len(bad_songs)):
    bad_ids.append(bad_songs[i]['track']['id'])
print(len(bad_ids))
print(bad_ids)

217
['5KVciTEfXvpcFHR3RFbtJy', '6iyp6udVpLuq3MCUulZT4h', '6YyvzSFCx1qogS5VWfXc6B', '7iaw359G2XT14uTfV9feip', '51rXHuKN8Loc4sUlKPODgH', '42XnD59SFcomCJu55EPhqJ', '5tFep7dXGd7vEJ668wTPux', '40oKW22ZNNkEdZLJTScaQI', '7DuvVy7JfFgXwWlVzyyPys', '0ESJlaM8CE1jRWaNtwSNj8', '4bJX21zvsGJELwVNf902qk', '07pJdL0L9XDGW0HBj0rIS2', '1cTZMwcBJT0Ka3UJPXOeeN', '6urLjX35oXXRYtcH1b3dCh', '6HmWy7ttkeZGG2KEtt8AqX', '1yxgsra98r3qAtxqiGZPiX', '3be3kLKYYLtAb8WERUM1sS', '4DTpngLjoHj5gFxEZFeD3J', '4qKcDkK6siZ7Jp1Jb4m0aL', '24lORMRGMv9sXpZJdN1PVm', '6Vur2D5ks3pdAZqqmLIeab', '2eMwDehkIC1j68U6FA3Eiq', '2yUbCEiaolfSMluDo9RMmG', '2nYwV62UABTH0NanrwVIdQ', '263le58t0LrdRlCZDcGQXd', '795qeY9x7NmfYJOZ9dQY0Y', '1xfFPJdRjgDAaMcDgbXwyh', '08Tbfp1GXta9Gb4RhoTuoX', '0H8XeaJunhvpBdBFIYi6Sh', '4fndbjoz1qJyK6JcLdKfzm', '5jyyPsIGM2yqkZN9R3TmvN', '4vIhvFLrnDD9fEjpjWs0Ky', '2D2w9943rsnJOGCrI4aMQp', '628Ueb4sRiXApObhcU9iPU', '1pF3VcWYHeuVPflRwGBaWV', '2Bv0MbaqYx2IrQa7Z6xBT0', '2Jlx7CGCVi3mRbKcA53YOn', '1cZlBZwnwGPtYeRIeQcoFh', '4TirSu

In [ ]:
source_playlist = sp.user_playlist(username, source_playlist_id)
source_tracks = source_playlist["tracks"]
source_songs = source_tracks["items"]

while source_tracks['next']:
    source_tracks = sp.next(source_tracks)
    for item in source_tracks["items"]:
        source_songs.append(item)

source_ids = []
for i in range(len(source_songs)):
    if source_songs[i]['track'] != None:
      source_ids.append(source_songs[i]['track']['id'])
print(len(source_ids))
print(source_ids)

122
['0Dm43YLUlGdePpSbI1ct8h', '70eFcWOvlMObDhURTqT4Fv', '1DhRbox3xkceP64k3JeYfW', '3FHXUqaUer9o231Je1zxIN', '2md2i5QvelRFnafpnd6LOg', '1z3L82jeYnTmYUjPc7zXcu', '1MJ5f5EYBC92ADD6xcz7nb', '0W0R48uImgPVVJgb5YTC3I', '0nMqu5vN44GNv8UOQxmBJK', '5pgZpHqfv4TSomtkfGZGrG', '14rWEXQGBsFB7ALUc8G4sr', '4sqIBV3ukQDqRr8Me2nvQb', '1QvpsbjYAVVFvFrWVC0qnQ', '6Wz9rIfo9tOBcVCd1Mo7F7', '3OUyyDN7EZrL7i0Sbi5SVd', '32q9OuCp7wFdLZKuCXm0wl', '5JWPUEov2wlX7c0jhYZpeB', '2BITQ360Knh6qNAOqR7Dyq', '5O2Yjlcs5pSpHq1gcA0cwg', '23bOfmnZiN42pB1frXNyak', '1LwwZ0xoUz5gR8XrxEUKAK', '17aVK3oQdgwVG2l0HpP67W', '25nzKGDiua1lE9Qo5V19GL', '60dwJ8dnqZ4WrIdiu7LkNR', '7KrlNS2VqiNNJ82RMQdxBn', '2xdwCJLfKCkat14mlgR26M', '3Jbpo2slhW2bbVneiJnC9G', '4dVbhS6OiYvFikshyaQaCN', '5UWwZ5lm5PKu6eKsHAGxOk', '67vYdAAM3oGsEImCRfbtsF', '3ptB4SmuSJTGhQByFrMJVJ', '2CDVqsGGkf2v1tMETt3fq5', '4QMSoa7Fh3y4kmMNuBtOgt', '1F9kEuwT29fgqlovIclu81', '0EC0kuRKIUlru5eRxWYYyv', '6KYVWcOQWm7JrlsPMkrkNi', '732Sqj2piYPhdMPENP4axz', '0ooz46SAYMJsdOPnwW4nKK', '1mSClO

In [ ]:
training_features = []

for i in range(0, len(good_ids), 50):
    audio_features = sp.audio_features(good_ids[i : i + 50])
    for track in audio_features:
        training_features.append(track)
        training_features[-1]['target'] = 1

for i in range(0, len(bad_ids), 50):
    audio_features = sp.audio_features(bad_ids[i : i + 50])
    for track in audio_features:
        training_features.append(track)
        training_features[-1]['target'] = 0

print(len(training_features))
print(training_features)

349
[{'danceability': 0.482, 'energy': 0.786, 'key': 7, 'loudness': -5.735, 'mode': 1, 'speechiness': 0.032, 'acousticness': 0.00321, 'instrumentalness': 8.88e-05, 'liveness': 0.446, 'valence': 0.793, 'tempo': 126.755, 'type': 'audio_features', 'id': '4DMKwE2E2iYDKY01C335Uw', 'uri': 'spotify:track:4DMKwE2E2iYDKY01C335Uw', 'track_href': 'https://api.spotify.com/v1/tracks/4DMKwE2E2iYDKY01C335Uw', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4DMKwE2E2iYDKY01C335Uw', 'duration_ms': 323000, 'time_signature': 4, 'target': 1}, {'danceability': 0.377, 'energy': 0.681, 'key': 7, 'loudness': -8.039, 'mode': 1, 'speechiness': 0.0298, 'acousticness': 0.00088, 'instrumentalness': 0.0023, 'liveness': 0.0504, 'valence': 0.285, 'tempo': 108.789, 'type': 'audio_features', 'id': '1QEEqeFIZktqIpPI4jSVSF', 'uri': 'spotify:track:1QEEqeFIZktqIpPI4jSVSF', 'track_href': 'https://api.spotify.com/v1/tracks/1QEEqeFIZktqIpPI4jSVSF', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1QEEqeFI

In [ ]:
source_features = []
for i in range(0, len(source_ids), 50):
    audio_features = sp.audio_features(source_ids[i : i + 50])
    for track in audio_features:
        source_features.append(track)
        source_features[-1]['target'] = 0  # arbitrary

print(len(source_features))
print(source_features)

122
[{'danceability': 0.662, 'energy': 0.624, 'key': 10, 'loudness': -13.606, 'mode': 0, 'speechiness': 0.0417, 'acousticness': 0.165, 'instrumentalness': 0, 'liveness': 0.0674, 'valence': 0.961, 'tempo': 174.513, 'type': 'audio_features', 'id': '0Dm43YLUlGdePpSbI1ct8h', 'uri': 'spotify:track:0Dm43YLUlGdePpSbI1ct8h', 'track_href': 'https://api.spotify.com/v1/tracks/0Dm43YLUlGdePpSbI1ct8h', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0Dm43YLUlGdePpSbI1ct8h', 'duration_ms': 252560, 'time_signature': 4, 'target': 0}, {'danceability': 0.64, 'energy': 0.647, 'key': 5, 'loudness': -8.114, 'mode': 0, 'speechiness': 0.186, 'acousticness': 0.124, 'instrumentalness': 0, 'liveness': 0.0801, 'valence': 0.546, 'tempo': 92.979, 'type': 'audio_features', 'id': '70eFcWOvlMObDhURTqT4Fv', 'uri': 'spotify:track:70eFcWOvlMObDhURTqT4Fv', 'track_href': 'https://api.spotify.com/v1/tracks/70eFcWOvlMObDhURTqT4Fv', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/70eFcWOvlMObDhURTqT4Fv'

## Data Preparation

In [ ]:
trainingData = pd.DataFrame(training_features)

In [ ]:
trainingData

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,target
0,0.482,0.786,7,-5.735,1,0.0320,0.00321,0.000089,0.4460,0.7930,126.755,audio_features,4DMKwE2E2iYDKY01C335Uw,spotify:track:4DMKwE2E2iYDKY01C335Uw,https://api.spotify.com/v1/tracks/4DMKwE2E2iYD...,https://api.spotify.com/v1/audio-analysis/4DMK...,323000,4,1
1,0.377,0.681,7,-8.039,1,0.0298,0.00088,0.002300,0.0504,0.2850,108.789,audio_features,1QEEqeFIZktqIpPI4jSVSF,spotify:track:1QEEqeFIZktqIpPI4jSVSF,https://api.spotify.com/v1/tracks/1QEEqeFIZktq...,https://api.spotify.com/v1/audio-analysis/1QEE...,285133,4,1
2,0.333,0.927,9,-8.550,0,0.0733,0.00290,0.000208,0.2970,0.3850,141.466,audio_features,5QTxFnGygVM4jFQiBovmRo,spotify:track:5QTxFnGygVM4jFQiBovmRo,https://api.spotify.com/v1/tracks/5QTxFnGygVM4...,https://api.spotify.com/v1/audio-analysis/5QTx...,308120,4,1
3,0.316,0.320,9,-13.045,1,0.0387,0.40800,0.000067,0.0925,0.2940,74.163,audio_features,05GhiSRMv34B1N6BsixbmL,spotify:track:05GhiSRMv34B1N6BsixbmL,https://api.spotify.com/v1/tracks/05GhiSRMv34B...,https://api.spotify.com/v1/audio-analysis/05Gh...,250933,3,1
4,0.341,0.293,9,-12.408,1,0.0356,0.75600,0.000023,0.0833,0.1780,93.294,audio_features,2V0yBHMWB8GoMm80AXH4ot,spotify:track:2V0yBHMWB8GoMm80AXH4ot,https://api.spotify.com/v1/tracks/2V0yBHMWB8Go...,https://api.spotify.com/v1/audio-analysis/2V0y...,249213,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,0.567,0.463,6,-11.669,0,0.5040,0.73200,0.000000,0.7090,0.3240,126.982,audio_features,40IQooNkPRVtn4zlE3ZFpA,spotify:track:40IQooNkPRVtn4zlE3ZFpA,https://api.spotify.com/v1/tracks/40IQooNkPRVt...,https://api.spotify.com/v1/audio-analysis/40IQ...,496173,4,0
345,0.554,0.498,9,-8.866,1,0.0685,0.23000,0.000006,0.7950,0.4130,79.635,audio_features,5hVghJ4KaYES3BFUATCYn0,spotify:track:5hVghJ4KaYES3BFUATCYn0,https://api.spotify.com/v1/tracks/5hVghJ4KaYES...,https://api.spotify.com/v1/audio-analysis/5hVg...,190067,4,0
346,0.598,0.252,2,-15.086,1,0.0644,0.19400,0.133000,0.0992,0.0352,120.064,audio_features,4S7YHmlWwfwArgd8LfSPud,spotify:track:4S7YHmlWwfwArgd8LfSPud,https://api.spotify.com/v1/tracks/4S7YHmlWwfwA...,https://api.spotify.com/v1/audio-analysis/4S7Y...,238307,4,0
347,0.621,0.601,6,-5.616,0,0.1480,0.05220,0.000000,0.4600,0.4570,116.735,audio_features,696DnlkuDOXcMAnKlTgXXK,spotify:track:696DnlkuDOXcMAnKlTgXXK,https://api.spotify.com/v1/tracks/696DnlkuDOXc...,https://api.spotify.com/v1/audio-analysis/696D...,163636,5,0


In [ ]:
train, test = train_test_split(trainingData, test_size = 0.15)

In [ ]:
target_features = ["danceability", "loudness", "valence", "energy", "instrumentalness", "acousticness", "key", "speechiness", "duration_ms"]

x_train = train[target_features]
y_train = train["target"]
x_test = test[target_features]
y_test = test["target"]

In [ ]:
x_train

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,duration_ms
232,0.570,-7.562,0.2700,0.703,0.000000,0.020000,6,0.2220,180016
188,0.695,-8.627,0.2100,0.628,0.219000,0.337000,5,0.1450,244653
63,0.649,-7.127,0.6760,0.811,0.009060,0.003910,7,0.0269,355400
281,0.554,-12.404,0.0612,0.286,0.031300,0.766000,0,0.1290,141173
125,0.529,-6.908,0.1880,0.843,0.000827,0.062800,6,0.0616,310333
...,...,...,...,...,...,...,...,...,...
201,0.452,-7.848,0.3650,0.559,0.000002,0.048100,6,0.0453,209480
145,0.774,-4.479,0.2270,0.529,0.000000,0.067400,5,0.0532,228431
252,0.765,-9.684,0.1380,0.497,0.000000,0.013000,0,0.2930,232864
107,0.319,-5.315,0.2280,0.952,0.154000,0.000063,4,0.0723,441467


In [ ]:
y_train

232    0
188    0
63     1
281    0
125    1
      ..
201    0
145    0
252    0
107    1
216    0
Name: target, Length: 296, dtype: int64

In [ ]:
x_test

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,duration_ms
12,0.337,-7.674,0.404,0.684,0.000007,0.031900,2,0.0495,255600
223,0.920,-6.030,0.729,0.881,0.000065,0.063700,1,0.1290,174347
256,0.909,-6.063,0.471,0.517,0.000002,0.036500,1,0.1290,113006
343,0.755,-5.585,0.678,0.772,0.000000,0.128000,6,0.4000,141827
41,0.684,-12.590,0.784,0.481,0.000066,0.475000,2,0.0384,250000
274,0.654,-5.486,0.457,0.763,0.000018,0.111000,5,0.0638,180539
307,0.442,-5.046,0.210,0.737,0.000000,0.058900,7,0.1090,218320
336,0.684,-10.155,0.355,0.486,0.000148,0.002680,1,0.1320,219893
7,0.331,-11.630,0.260,0.183,0.000009,0.936000,5,0.0287,217400
100,0.300,-5.399,0.756,0.936,0.021000,0.033700,2,0.0456,271907


In [ ]:
y_test

12     1
223    0
256    0
343    0
41     1
274    0
307    0
336    0
7      1
100    1
32     1
119    1
212    0
85     1
81     1
22     1
105    1
111    1
21     1
229    0
218    0
149    0
191    0
319    0
160    0
259    0
260    0
242    0
273    0
167    0
53     1
37     1
296    0
181    0
156    0
72     1
143    0
25     1
261    0
150    0
326    0
332    0
169    0
190    0
272    0
116    1
77     1
117    1
6      1
31     1
3      1
68     1
234    0
Name: target, dtype: int64

## Model Evaluation

In [ ]:
# linear regression is a good start, but does not work well for classification problems
linreg = LinearRegression()
linreg.fit(x_train, y_train)
linreg_pred = linreg.predict(x_test)

print('Mean squared error: %.2f' % mean_squared_error(y_test, linreg_pred))
linreg_pred

Mean squared error: 0.07


array([ 0.81468396,  0.19710193, -0.15837265, -0.06123986,  0.64003271,
        0.35897535,  0.44818795,  0.27242119,  0.5698114 ,  1.08441786,
        0.62777459,  0.89799161,  0.35145299,  0.83530745,  0.91333017,
        1.5073824 ,  1.10995052,  0.60636466,  0.91536799,  0.19299615,
        0.21227654,  0.21961212,  0.24300545,  0.18749531,  0.40860803,
       -0.04772168, -0.06216331,  0.12885933, -0.13593773, -0.02489438,
        0.84739788,  0.77381493,  0.5167257 , -0.08802549, -0.12156916,
        1.01842579, -0.28878411,  0.63775297, -0.02471336, -0.13724872,
       -0.23361647, -0.1305064 ,  0.22370965, -0.33162798,  0.11895936,
        0.92331646,  0.64916877,  0.58011713,  0.83676882,  1.06294928,
        0.75683998,  0.7401017 ,  0.44585933])

In [ ]:
# k-neighbors algorithm is popular but not so applicable in this case, no local structure to data
knn = KNeighborsClassifier(3)
knn.fit(x_train, y_train)
knn_pred = knn.predict(x_test)

score = accuracy_score(y_test, knn_pred) * 100
print("Accuracy using Knn Tree: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, knn_pred))

Accuracy using Knn Tree:  62.26 %
Mean squared error: 0.38


In [ ]:
# decision tree is a common element in machine learning, useful in classification
dtc = DecisionTreeClassifier(min_samples_split=100)
dtc.fit(x_train, y_train)
dtc_pred = dtc.predict(x_test)

score = accuracy_score(y_test, dtc_pred) * 100
print("Accuracy using Decision Tree: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, dtc_pred))

Accuracy using Decision Tree:  94.34 %
Mean squared error: 0.06


In [ ]:
# random forest combines multiple decision trees and uses bagging
# (training multiple trees on different sections of training data, averaging the result)
rfc = RandomForestClassifier(n_jobs=1, random_state=1)
rfc.fit(x_train, y_train)
rfc_pred = rfc.predict(x_test)

score = accuracy_score(y_test, rfc_pred) * 100
print("Accuracy using Random Forest: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, rfc_pred))

Accuracy using Random Forest:  100.0 %
Mean squared error: 0.00


In [ ]:
# ada boost combines multiple decision trees and uses boosting
# (training a tree repeatedly on the result of previous training to reduce error in each subsequent tree)
ada = AdaBoostClassifier(n_estimators=100)
ada.fit(x_train, y_train)
ada_pred = ada.predict(x_test)

score = accuracy_score(y_test, ada_pred) * 100
print("Accuracy using Ada Boost: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, ada_pred))

Accuracy using Ada Boost:  98.11 %
Mean squared error: 0.02


In [ ]:
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=.1, max_depth=1, random_state=2)
gbc.fit(x_train, y_train)
gbc_pred = gbc.predict(x_test)

score = accuracy_score(y_test, gbc_pred) * 100
print("Accuracy using Gradient Boost: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, gbc_pred))

Accuracy using Gradient Boost:  96.23 %
Mean squared error: 0.04


In [ ]:
model = ada

## Generate Predictions

In [ ]:
sourceData = pd.DataFrame(source_features)
sourceData = sourceData[target_features]

In [ ]:
sourceData

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,duration_ms
0,0.662,-13.606,0.9610,0.624,0.000000,0.16500,10,0.0417,252560
1,0.640,-8.114,0.5460,0.647,0.000000,0.12400,5,0.1860,197867
2,0.716,-5.313,0.9100,0.823,0.000000,0.13200,5,0.0495,161747
3,0.468,-6.536,0.7620,0.630,0.000000,0.31000,8,0.0843,204507
4,0.397,-4.577,0.5560,0.903,0.000002,0.00575,2,0.0457,201373
...,...,...,...,...,...,...,...,...,...
117,0.519,-9.982,0.4880,0.181,0.000000,0.94400,5,0.0795,82147
118,0.615,-8.615,0.7470,0.592,0.955000,0.04880,0,0.0443,263673
119,0.512,-5.128,0.8380,0.955,0.000013,0.17200,7,0.3540,40227
120,0.807,-9.729,0.9610,0.510,0.115000,0.04100,0,0.0433,279773


In [ ]:
pred = model.predict(sourceData)

In [ ]:
source_playlist_songs = source_playlist['tracks']['items']

print("ACCEPTED")

i = 0
for prediction in pred:
    if prediction == 1:
        song_title = source_playlist_songs[i + 1]['track']['name']
        artist_name = source_playlist_songs[i + 1]['track']['artists'][0]['name']
        print ("Song: " + song_title + ", By: "+ artist_name)
    i = i + 1


print("REJECTED")

i = 0
for prediction in pred:
    if prediction == 0:
        song_title = source_playlist_songs[i + 1]['track']['name']
        artist_name = source_playlist_songs[i + 1]['track']['artists'][0]['name']
        print ("Song: " + song_title + ", By: "+ artist_name)
    i = i + 1

ACCEPTED
Song: Part-Time Lover, By: Stevie Wonder
Song: Meant to Live, By: Switchfoot
Song: Meant To Live - Acoustic Version, By: Switchfoot
Song: Chamber Of Reflection, By: Mac DeMarco
Song: Wavy, By: Janko Nilovic
Song: Malibu 1992, By: COIN
Song: Revolution 909, By: Daft Punk
Song: Soap On Your Skin, By: The Solids
Song: Across The Overpass, By: The Solids
Song: Hey Beautiful - From "How I Met Your Mother"/Theme, By: The Solids
Song: BYE, By: Jaden
Song: Lost In Yesterday, By: Tame Impala
Song: Starchild, By: Burn Pilot
Song: Young Blood, By: The Naked And Famous
Song: Saturday, By: Twenty One Pilots
Song: My Hero, By: Foo Fighters
Song: Everlong, By: Foo Fighters
Song: Times Like These, By: Foo Fighters
Song: the angel of 8th ave., By: Gang of Youths
Song: love race (feat. Kellin Quinn), By: Machine Gun Kelly
Song: Enghave, By: Marco Egeberg
Song: Relax, By: Frankie Goes To Hollywood
Song: Rockstar, By: Mallrat
Song: Galaxies, By: Laura Veirs
Song: Over Now (with The Weeknd), By: C